# Boston Housing Classification Random Forest

In [ ]:
import sys
sys.path.append("..")
from pyspark.sql.types import BooleanType
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import expr
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from helpers.path_translation import translate_to_file_string
from helpers.data_prep_and_print import print_df

In [ ]:
inputFile = "hdfs:///data/Boston_Housing_Data.csv"
# inputFile = translate_to_file_string("../data/Boston_Housing_Data.csv")

Spark session creation 

In [ ]:
spark = (SparkSession
       .builder
       .master("yarn") 
       .appName("BostonHousingClassificationRandomForest")
       .getOrCreate())

DataFrame creation using an ifered Schema 

In [ ]:
df = spark.read.option("header", "true") \
       .option("inferSchema", "true") \
       .option("delimiter", ";") \
       .csv(inputFile) \
       .withColumn("CATBOOL", expr("CAT").cast(BooleanType()))
print(df.printSchema())

Prepare training and test data.

In [ ]:
featureCols = df.columns.copy()
featureCols.remove("MEDV")
featureCols.remove("CAT")
featureCols.remove("CATBOOL") 
print(featureCols)

assembler =  VectorAssembler(outputCol="features", inputCols=featureCols)

In [ ]:
labledPointDataSet = assembler.transform(df)
splits = labledPointDataSet.randomSplit([0.9, 0.1 ], 12345)
training = splits[0]
test = splits[1]

Random Forest Classifier

In [ ]:
rf = RandomForestClassifier(labelCol="CAT", featuresCol="features",impurity="gini", \
                 minInstancesPerNode=10, featureSubsetStrategy='sqrt', subsamplingRate=0.95, seed= 12345)

Train the model 

In [ ]:
rfModel = rf.fit(training)

Test the model

In [ ]:
predictions = rfModel.transform(test)
print_df(predictions)

In [ ]:
evaluator = BinaryClassificationEvaluator(labelCol="CAT",rawPredictionCol="rawPrediction", metricName="areaUnderROC")
accuracy = evaluator.evaluate(predictions)
print("Test Error",(1.0 - accuracy))